# Dependencias

In [1]:
%pip install pandas
%pip install sklearn
%pip install matplotlib
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 285.1/285.1 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Librerias

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# Label Mappings
| a | b | c | d | e | f | g | h | i | k  | l  | m  | n  | o  | p  | q  | r  | s  | t  | u  | v  | w  | x  | y  | z |
|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 |

In [2]:
mappings = np.array(["a","b","c","d","e","f","g","h","i","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"])

# plotear matrix de confusion

In [2]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_cnf_matrix(Y_test, Y_pred):
  cnf_matrix = confusion_matrix(Y_test, Y_pred)

  cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]

  plt.rcParams['figure.figsize'] = (20, 10)
  ax= plt.subplot()
  sns.heatmap(cnf_matrix, annot=True, fmt='.2f', ax=ax);  
  
  ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
  ax.set_title('Confusion Matrix'); 

# Cargar datos

In [2]:
sign_df = pd.read_csv("dataset/sign_mnist_full.csv")

sign_X = sign_df.loc[:,  sign_df.columns != "label"].values
sign_Y = sign_df.loc[:,["label"]].values

In [5]:
sign_df_train = pd.read_csv("dataset/sign_mnist_train.csv")

sign_X_train = sign_df_train.loc[:,  sign_df_train.columns != "label"].values
sign_Y_train = sign_df_train.loc[:,["label"]].values

sign_df_test = pd.read_csv("dataset/sign_mnist_test.csv")

sign_X_test = sign_df_test.loc[:,  sign_df_test.columns != "label"].values
sign_Y_test = sign_df_test.loc[:,["label"]].values

# K fold generalizado

In [32]:
from xml.etree.ElementInclude import include
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score
from sklearn.metrics import classification_report
import numpy as np
import itertools

def KFold_CLF(X_, Y_, k_splits_ = 10, shuffle_ = False, model = None, reducer = None):
  kf = KFold(n_splits=k_splits_, shuffle=shuffle_, random_state=1313)
  precision = recall = f1 = AUC = 0
  i = 0
  for train_index, test_index in kf.split(X_):
    # Seleccion train test splits para este KFold
    X_train, X_test = X_[train_index, :], X_[test_index, :] 
    Y_train, Y_test = Y_[train_index], Y_[test_index]

    # Normalizacion de datos
    X_train = X_train / 255
    X_train = pd.DataFrame(X_train)
    X_test = X_test / 255
    X_test = pd.DataFrame(X_test)

    # reducir dimensionalidad, mapea a n_comps_ vectores
    if reducer == None:
      X_train_reducer = X_train
      X_test_reducer = X_test
    else:
      X_train_reducer = reducer.fit_transform(X_train,Y_train.ravel())
      X_test_reducer = reducer.transform(X_test)

    # Fit, Predict
    model.fit(X_train_reducer, Y_train.ravel())
    # print("model fitted")
    
    Y_predicted = model.predict(X_test_reducer)
    # print("model predicted")

    print(Y_test.shape)
    print(Y_predicted.shape)
    
    # metrics
    precision += precision_score(Y_test, Y_predicted, average="macro")
    recall += recall_score(Y_test, Y_predicted, average="macro")
    f1 += f1_score(Y_test, Y_predicted,  average="macro")
    AUC += roc_auc_score(Y_test, model.predict_proba(X_test_reducer),multi_class='ovr',average='macro')
    print(f"{i}...",end='')
    i += 1

  print("\n")
  precision /= k_splits_
  recall /= k_splits_
  f1 /= k_splits_
  AUC /= k_splits_

  print(f"Final metrics ({k_splits_}-Folds, shuffling={shuffle_})\n", 
  "Precision:", "{:.2f}".format(precision),
  " | Recall:", "{:.2f}".format(recall), 
  " | F1:", "{:.2f}".format(f1),
  " | AUC:","{:.2f}".format(AUC))

  # plot_cnf_matrix(Y_test, Y_predicted)
  return(np.array([[precision, recall, f1, AUC]]))

## K fold LDA, pruebas de hiperparámetros

In [40]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier

cols = ['Precision', 'Recall', 'F1', 'AUC', 'Validator(...)','Reductor(...)', 'Classifier(...)']
table_res_kf = pd.DataFrame(columns=cols)

# KFOLD ------

# K neighbors LDA
print("KNN Classification")
for k_f in [5]:
  for sh in [True]:
    for vector in [20]:
      for K_ in [1,3,6]:
        for W_ in ['uniform','distance']:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                              k_splits_=k_f, 
                              shuffle_=sh,  
                              model=KNeighborsClassifier(n_neighbors=K_, weights=W_),
                              reducer=LDA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                              [f"KFold(n_iterations={k_f}, shuffle={sh})",
                                f"LDA(n_components={vector})",
                                f"KNeighborsClassifier(n_neighbors={K_},weights={W_})"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_knn.csv",index=False) #write

# SVM LDA
print("SVM Classification rbf")
for k_f in [5]:
  for sh in [True]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        for G_ in [0.01, 0.1]:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                              k_splits_=k_f, 
                              shuffle_=sh,  
                              model=SVC(kernel='rbf', gamma=G_, decision_function_shape='ovo',
                                        C=C_, probability=True),
                              reducer=LDA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                              [f"KFold(n_iterations={k_f}, shuffle={sh})",
                                f"LDA(n_components={vector})",
                                f"SVC(kernel='rbf',C={C_},gamma={G_},decision_function_shape=ovo)"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write

print("SVM Classification poly")
for k_f in [5]:
  for sh in [True]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = KFold_CLF(sign_X_train, sign_Y_train, 
                            k_splits_=k_f, 
                            shuffle_=sh,  
                            model=SVC(kernel='poly', decision_function_shape='ovo',
                                      C=C_, probability=True),
                            reducer=LDA(n_components=vector))
        table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"LDA(n_components={vector})",
                              f"SVC(kernel='poly',C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write

print("SVM Classification linear")
for k_f in [5]:
  for sh in [True]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = KFold_CLF(sign_X_train, sign_Y_train, 
                            k_splits_=k_f, 
                            shuffle_=sh,  
                            model=CalibratedClassifierCV(LinearSVC(C=C_,
                            multi_class='ovr',max_iter=10000)),
                            reducer=LDA(n_components=vector))
        table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"LDA(n_components={vector})",
                              f"LinearSVC(C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write
        
# Decision trees
print("Decision Tree Classification")
for k_f in [5]:
  for sh in [True]:
    for vector in [20]:
      for depth in [5,20,None]:
        for split_ in [2,10,20]:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                          k_splits_=k_f, 
                          shuffle_=sh,  
                          model = DecisionTreeClassifier(criterion="gini",
                          min_samples_split=split_, max_depth=depth),
                          reducer=LDA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"LDA(n_components={vector})",
                              f"DecisionTreeClassifier(criterion='gini',min_samples_split={split_},max_depth={depth})"])],
                            columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_dt.csv",index=False) #write


Decision Tree Classification
(5491, 1)
(5491,)
0...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
1...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
2...

...

(5491, 1)
(5491,)
3...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.26  | Recall: 0.28  | F1: 0.21  | AUC: 0.83


C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.96  | Recall: 0.96  | F1: 0.96  | AUC: 0.99
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.96  | Recall: 0.96  | F1: 0.96  | AUC: 0.99
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.95  | Recall: 0.95  | F1: 0.95  | AUC: 0.99
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.97  | Recall: 0.97  | F1: 0.97  | AUC: 0.98
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precisi

## K fold PCA, pruebas de hiperparametros

In [42]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier

cols = ['Precision', 'Recall', 'F1', 'AUC', 'Validator(...)','Reductor(...)', 'Classifier(...)']
table_res_kf = pd.DataFrame(columns=cols)

# KFOLD ------

# K neighbors PCA
print("KNN Classification")
for k_f in [5]:
  for sh in [True]:
    for vector in [50]:
      for K_ in [1,3,6]:
        for W_ in ['uniform','distance']:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                              k_splits_=k_f, 
                              shuffle_=sh,  
                              model=KNeighborsClassifier(n_neighbors=K_, weights=W_),
                              reducer=PCA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                              [f"KFold(n_iterations={k_f}, shuffle={sh})",
                                f"PCA(n_components={vector})",
                                f"KNeighborsClassifier(n_neighbors={K_},weights={W_})"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_knn.csv",index=False) #write

# SVM PCA
print("SVM Classification rbf")
for k_f in [5]:
  for sh in [True]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        for G_ in [0.01]:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                              k_splits_=k_f, 
                              shuffle_=sh,  
                              model=SVC(kernel='rbf', gamma=G_, decision_function_shape='ovo',
                                        C=C_, probability=True),
                              reducer=PCA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                              [f"KFold(n_iterations={k_f}, shuffle={sh})",
                                f"PCA(n_components={vector})",
                                f"SVC(kernel='rbf',C={C_},gamma={G_},decision_function_shape=ovo)"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write

print("SVM Classification poly")
for k_f in [5]:
  for sh in [True]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = KFold_CLF(sign_X_train, sign_Y_train, 
                            k_splits_=k_f, 
                            shuffle_=sh,  
                            model=SVC(kernel='poly', decision_function_shape='ovo',
                                      C=C_, probability=True),
                            reducer=PCA(n_components=vector))
        table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"PCA(n_components={vector})",
                              f"SVC(kernel='poly',C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write

print("SVM Classification linear")
for k_f in [5]:
  for sh in [True]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = KFold_CLF(sign_X_train, sign_Y_train, 
                            k_splits_=k_f, 
                            shuffle_=sh,  
                            model=CalibratedClassifierCV(LinearSVC(C=C_,
                            multi_class='ovr',max_iter=10000)),
                            reducer=PCA(n_components=vector))
        table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"PCA(n_components={vector})",
                              f"LinearSVC(C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_kf.to_csv("table_res_kf_svm.csv",index=False) #write
        
print("Decision Tree Classification")
for k_f in [5]:
  for sh in [True]:
    for vector in [50]:
      for depth in [5,20,None]:
        for split_ in [2,10,20]:
          res = KFold_CLF(sign_X_train, sign_Y_train, 
                          k_splits_=k_f, 
                          shuffle_=sh,  
                          model = DecisionTreeClassifier(criterion="gini",
                          min_samples_split=split_, max_depth=depth),
                          reducer=PCA(n_components=vector))
          table_res_kf = pd.concat([table_res_kf, pd.DataFrame([np.append(res, 
                            [f"KFold(n_iterations={k_f}, shuffle={sh})",
                              f"PCA(n_components={vector})",
                              f"DecisionTreeClassifier(criterion='gini',min_samples_split={split_},max_depth={depth})"])],
                            columns=cols)],axis=0, ignore_index = True)
          table_res_kf.to_csv("table_res_kf_dt.csv",index=False) #write


Decision Tree Classification
(5491, 1)
(5491,)
0...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
1...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
2...

...

(5491, 1)
(5491,)
3...

C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.35  | Recall: 0.28  | F1: 0.25  | AUC: 0.82


C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.95  | Recall: 0.95  | F1: 0.95  | AUC: 0.98
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.93  | Recall: 0.93  | F1: 0.93  | AUC: 0.98
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.91  | Recall: 0.91  | F1: 0.91  | AUC: 0.98
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precision: 0.96  | Recall: 0.96  | F1: 0.96  | AUC: 0.98
(5491, 1)
(5491,)
0...(5491, 1)
(5491,)
1...(5491, 1)
(5491,)
2...(5491, 1)
(5491,)
3...(5491, 1)
(5491,)
4...

Final metrics (5-Folds, shuffling=True)
 Precisi

# Bootstrap generalizado

In [7]:
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score


def Bootstrap_CLF(X_, Y_, n_iterations = 10, sample_size = 0.50, model = None, reducer = None):
  n_size = int(X_.shape[0] * sample_size)
  indexes = list(range(0, X_.shape[0]))
  precision = recall = f1 = AUC = 0

  for i in range(n_iterations):
    # Seleccion bootstrap
    train = resample(indexes, n_samples = n_size)
    train_unique = np.unique(train)
    test = [ele for ele in indexes if ele not in train_unique]
    X_train = X_[train]
    Y_train = Y_[train]
    X_test = X_[test]
    Y_test = Y_[test]

    # Normalizacion de datos
    X_train = X_train / 255
    X_train = pd.DataFrame(X_train)
    X_test = X_test / 255
    X_test = pd.DataFrame(X_test)

    # reducir dimensionalidad, mapea a n_comps_ vectores
    if reducer == None:
      X_train_reducer = X_train
      X_test_reducer = X_test
    else:
      X_train_reducer = reducer.fit_transform(X_train,Y_train.ravel())
      X_test_reducer = reducer.transform(X_test)

    # Fit, Predict
    model.fit(X_train_reducer, Y_train.ravel())
    # print("model fitted")

    Y_predicted = model.predict(X_test_reducer)
    # print("model predicted")

    print(Y_test.shape)
    print(Y_predicted.shape)
    
    # metrics
    precision += precision_score(Y_test, Y_predicted, average="macro")
    recall += recall_score(Y_test, Y_predicted, average="macro")
    f1 += f1_score(Y_test, Y_predicted,  average="macro")
    AUC += roc_auc_score(Y_test, model.predict_proba(X_test_reducer),multi_class='ovr',average='macro')
    print(f"{i}...",end='')
    
  print("\n")
  precision /= n_iterations
  recall /= n_iterations
  f1 /= n_iterations
  AUC /= n_iterations
  
  print(f"Final metrics ({n_iterations}-Iterations, {sample_size}-Sample Ratio)\n", 
  "Precision:", "{:.2f}".format(precision),
  " | Recall:", "{:.2f}".format(recall), 
  " | F1:", "{:.2f}".format(f1),
  " | AUC:","{:.2f}".format(AUC))
  # plot_cnf_matrix(Y_test, Y_predicted)
  return(np.array([[precision, recall, f1, AUC]]))

## Bootstrap LDA, pruebas de hiperparámetros

In [21]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier

cols = ['Precision', 'Recall', 'F1', 'AUC', 'Validator(...)','Reductor(...)', 'Classifier(...)']
table_res_bs = pd.DataFrame(columns=cols)

# KFOLD ------

# K neighbors LDA
print("KNN Classification")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for K_ in [1,3,6]:
        for W_ in ['uniform','distance']:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train, 
                              n_iterations=it, 
                              sample_size=sample,  
                              model=KNeighborsClassifier(n_neighbors=K_, weights=W_),
                              reducer=LDA(n_components=vector))
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                              [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                                f"LDA(n_components={vector})",
                                f"KNeighborsClassifier(n_neighbors={K_},weights={W_})"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_knn.csv",index=False) #write

# SVM LDA
print("SVM Classification rbf")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        for G_ in [0.01, 0.1]:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                              n_iterations=it, 
                              sample_size=sample,  
                              model=SVC(kernel='rbf', gamma=G_, decision_function_shape='ovo',
                                        C=C_, probability=True),
                              reducer=LDA(n_components=vector))
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                              [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                                f"LDA(n_components={vector})",
                                f"SVC(kernel='rbf',C={C_},gamma={G_},decision_function_shape=ovo)"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write

print("SVM Classification poly")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                            n_iterations=it, 
                            sample_size=sample,  
                            model=SVC(kernel='poly', decision_function_shape='ovo',
                                      C=C_, probability=True),
                            reducer=LDA(n_components=vector))
        table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"LDA(n_components={vector})",
                              f"SVC(kernel='poly',C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write

print("SVM Classification linear")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                            n_iterations=it, 
                            sample_size=sample,
                            model=CalibratedClassifierCV(LinearSVC(C=C_,
                            multi_class='ovr',max_iter=10000)),
                            reducer=LDA(n_components=vector))
        table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"LDA(n_components={vector})",
                              f"LinearSVC(C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write
        
# Decision trees
print("Decision Tree Classification")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for depth in [5,20,None]:
        for split_ in [2,10,20]:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train, 
                              n_iterations=it, 
                              sample_size=sample,  
                              model = DecisionTreeClassifier(criterion="gini",
                              min_samples_split=split_, max_depth=depth),
                              reducer=LDA(n_components=vector))
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"LDA(n_components={vector})",
                              f"DecisionTreeClassifier(criterion='gini',min_samples_split={split_},max_depth={depth})"])],
                            columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_dt.csv",index=False) #write  


SVM Classification
(20979, 1)
(20979,)


c:\Users\mauri\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0...

Final metrics (1-Iterations, 0.5-Sample Ratio)
 Precision: 0.71  | Recall: 0.46  | F1: 0.47  | AUC: 0.97


## Bootstrap PCA, pruebas de hiperparametros


In [ ]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier

cols = ['Precision', 'Recall', 'F1', 'AUC', 'Validator(...)','Reductor(...)', 'Classifier(...)']
table_res_bs = pd.DataFrame(columns=cols)

# KFOLD ------

# K neighbors PCA
print("KNN Classification")
for it in [5]:
  for sample in [0.80]:
    for vector in [50]:
      for K_ in [1,3,6]:
        for W_ in ['uniform','distance']:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train, 
                              n_iterations=it, 
                              sample_size=sample,  
                              model=KNeighborsClassifier(n_neighbors=K_, weights=W_),
                              reducer=PCA(n_components=vector))
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                              [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                                f"PCA(n_components={vector})",
                                f"KNeighborsClassifier(n_neighbors={K_},weights={W_})"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_knn.csv",index=False) #write

# SVM PCA
print("SVM Classification rbf")
for it in [5]:
  for sample in [0.80]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        for G_ in [0.01, 0.1]:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                              n_iterations=it, 
                              sample_size=sample,  
                              model=SVC(kernel='rbf', gamma=G_, decision_function_shape='ovo',
                                        C=C_, probability=True),
                              reducer=PCA(n_components=vector))
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                              [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                                f"PCA(n_components={vector})",
                                f"SVC(kernel='rbf',C={C_},gamma={G_},decision_function_shape=ovo)"])],
                              columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write

print("SVM Classification poly")
for it in [5]:
  for sample in [0.80]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                            n_iterations=it, 
                            sample_size=sample,  
                            model=SVC(kernel='poly', decision_function_shape='ovo',
                                      C=C_, probability=True),
                            reducer=PCA(n_components=vector))
        table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"PCA(n_components={vector})",
                              f"SVC(kernel='poly',C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write

print("SVM Classification linear")
for it in [5]:
  for sample in [0.80]:
    for vector in [50]:
      for C_ in [2,4,8]: # valores peque;os para dataset grande
        res = Bootstrap_CLF(sign_X_train, sign_Y_train,  
                            n_iterations=it, 
                            sample_size=sample,
                            model=CalibratedClassifierCV(LinearSVC(C=C_,
                            multi_class='ovr',max_iter=10000)),
                            reducer=PCA(n_components=vector))
        table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"PCA(n_components={vector})",
                              f"LinearSVC(C={C_},decision_function_shape=ovo)"])],
                            columns=cols)],axis=0, ignore_index = True)
        table_res_bs.to_csv("table_res_bs_svm.csv",index=False) #write
  
# Decision trees
print("Decision Tree Classification")
for it in [5]:
  for sample in [0.80]:
    for vector in [20]:
      for depth in [5,20,None]:
        for split_ in [2,10,20]:
          res = Bootstrap_CLF(sign_X_train, sign_Y_train, 
                              n_iterations=it, 
                              sample_size=sample,  

                              model = DecisionTreeClassifier(criterion="gini",
                              min_samples_split=split_, max_depth=depth),
                              reducer=PCA(n_components=vector))
                              
          table_res_bs = pd.concat([table_res_bs, pd.DataFrame([np.append(res, 
                            [f"Bootstrap(n_iterations={it}, sample_size={sample})",
                              f"PCA(n_components={vector})",
                              f"DecisionTreeClassifier(criterion='gini',min_samples_split={split_},max_depth={depth})"])],
                            columns=cols)],axis=0, ignore_index = True)
          table_res_bs.to_csv("table_res_bs_dt.csv",index=False) #write 


# Modelos Finales

In [59]:
pd.set_option('display.expand_frame_repr', False)

## KNN 

In [43]:
KNN_KF_LDA_df = pd.read_csv("finaldatas_LDA/table_res_kf_knn.csv")
KNN_BS_LDA_df = pd.read_csv("finaldatas_LDA/table_res_bs_knn.csv")
KNN_KF_PCA_df = pd.read_csv("finaldatas_PCA/table_res_kf_knn.csv")
KNN_BS_PCA_df = pd.read_csv("finaldatas_PCA/table_res_bs_knn.csv")

### LDA

In [61]:
KNN_KF_LDA_df # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.999892,0.999897,0.999894,0.999946,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=1,weights=uni..."
1,0.999892,0.999897,0.999894,0.999946,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=1,weights=dis..."
2,0.999685,0.999690,0.999687,0.999984,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=3,weights=uni..."
3,0.999685,0.999690,0.999687,0.999984,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=3,weights=dis..."
4,0.999208,0.999198,0.999201,0.999984,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=6,weights=uni..."
5,0.999406,0.999400,0.999402,0.999984,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=6,weights=dis..."


In [52]:
KNN_BS_LDA_df #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.999694,0.999698,0.999696,0.999842,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=1,weights=uni..."
1,0.999734,0.999722,0.999727,0.999855,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=1,weights=dis..."
2,0.999485,0.999475,0.999480,0.999915,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=3,weights=uni..."
3,0.999386,0.999388,0.999386,0.999925,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=3,weights=dis..."
4,0.999054,0.999052,0.999052,0.999984,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=6,weights=uni..."
5,0.999243,0.999232,0.999237,0.999944,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"KNeighborsClassifier(n_neighbors=6,weights=dis..."


### PCA

In [62]:
KNN_KF_PCA_df # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.998395,0.998423,0.998405,0.999177,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=1,weights=uni..."
1,0.998434,0.998456,0.998441,0.999194,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=1,weights=dis..."
2,0.994164,0.994257,0.994184,0.999589,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=3,weights=uni..."
3,0.996558,0.996557,0.996546,0.999576,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=3,weights=dis..."
4,0.986333,0.986431,0.986302,0.999715,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=6,weights=uni..."
5,0.994953,0.994942,0.994931,0.999747,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=6,weights=dis..."


In [63]:
KNN_BS_PCA_df #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.994353,0.994324,0.994324,0.997038,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=1,weights=uni..."
1,0.994397,0.994441,0.994408,0.997100,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=1,weights=dis..."
2,0.987808,0.987910,0.987825,0.998291,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=3,weights=uni..."
3,0.991003,0.991049,0.991007,0.998033,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=3,weights=dis..."
4,0.974276,0.974223,0.974098,0.998725,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=6,weights=uni..."
5,0.986457,0.986609,0.986481,0.998922,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"KNeighborsClassifier(n_neighbors=6,weights=dis..."


## SVM

In [64]:
SVM_KF_LDA_df = pd.read_csv("finaldatas_LDA/table_res_kf_svm.csv")
SVM_BS_LDA_df = pd.read_csv("finaldatas_LDA/table_res_bs_svm.csv")
SVM_KF_PCA_df = pd.read_csv("finaldatas_PCA/table_res_kf_svm.csv")
SVM_BS_PCA_df = pd.read_csv("finaldatas_PCA/table_res_bs_svm.csv")

### LDA

In [65]:
SVM_KF_LDA_df # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.997682,0.997727,0.997699,0.999994,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=2,gamma=0.01,decision_funct..."
1,0.998145,0.997840,0.997971,0.999999,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=2,gamma=0.1,decision_functi..."
2,0.998364,0.998344,0.998350,0.999997,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=4,gamma=0.01,decision_funct..."
3,0.998145,0.997840,0.997971,0.999999,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=4,gamma=0.1,decision_functi..."
4,0.998411,0.998409,0.998406,0.999997,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=8,gamma=0.01,decision_funct..."
5,0.998115,0.997804,0.997938,0.999999,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='rbf',C=8,gamma=0.1,decision_functi..."
6,0.989715,0.986597,0.987672,0.999900,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='poly',C=2,decision_function_shape=..."
7,0.993535,0.992376,0.992802,0.999960,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='poly',C=4,decision_function_shape=..."
8,0.995755,0.995224,0.995435,0.999979,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"SVC(kernel='poly',C=8,decision_function_shape=..."
9,0.986288,0.986169,0.986183,0.999663,"KFold(n_iterations=5, shuffle=True)",LDA(n_components=20),"LinearSVC(C=2,decision_function_shape=ovo)"


In [66]:
SVM_BS_LDA_df #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.997278,0.997289,0.997274,0.999991,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=2,gamma=0.01,decision_funct..."
1,0.996862,0.996255,0.996513,0.999997,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=2,gamma=0.1,decision_functi..."
2,0.998414,0.998406,0.998409,0.999997,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=4,gamma=0.01,decision_funct..."
3,0.996760,0.996221,0.996442,0.999997,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=4,gamma=0.1,decision_functi..."
4,0.998322,0.998285,0.998300,0.999997,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=8,gamma=0.01,decision_funct..."
5,0.996763,0.996070,0.996360,0.999998,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='rbf',C=8,gamma=0.1,decision_functi..."
6,0.989242,0.985406,0.986723,0.999917,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='poly',C=2,decision_function_shape=..."
7,0.993288,0.992101,0.992547,0.999927,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='poly',C=4,decision_function_shape=..."
8,0.995613,0.994969,0.995224,0.999966,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"SVC(kernel='poly',C=8,decision_function_shape=..."
9,0.981318,0.981341,0.981254,0.999431,"Bootstrap(n_iterations=5, sample_size=0.8)",LDA(n_components=20),"LinearSVC(C=2,decision_function_shape=ovo)"


### PCA

In [72]:
SVM_KF_PCA_df # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.976433,0.975882,0.975916,0.999769,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='rbf',C=2,gamma=0.01,decision_funct..."
1,0.993431,0.993113,0.993187,0.999985,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='rbf',C=4,gamma=0.01,decision_funct..."
2,0.999547,0.999517,0.999528,1.000000,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='rbf',C=8,gamma=0.01,decision_funct..."
3,0.989374,0.989077,0.989065,0.999920,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='poly',C=2,decision_function_shape=..."
4,0.995842,0.995886,0.995835,0.999951,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='poly',C=4,decision_function_shape=..."
5,0.997837,0.997868,0.997844,0.999952,"KFold(n_iterations=5, shuffle=True)",PCA(n_components=50),"SVC(kernel='poly',C=8,decision_function_shape=..."


In [71]:
SVM_BS_PCA_df #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Reductor(...),Classifier(...)
0,0.994038,0.993507,0.993724,0.999983,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='rbf',C=2,gamma=0.01,decision_funct..."
1,0.993634,0.993045,0.993286,0.999985,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='rbf',C=4,gamma=0.01,decision_funct..."
2,0.993537,0.992923,0.993166,0.999972,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='rbf',C=8,gamma=0.01,decision_funct..."
3,0.978188,0.976810,0.977011,0.999502,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='poly',C=2,decision_function_shape=..."
4,0.989086,0.988876,0.988900,0.999767,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='poly',C=4,decision_function_shape=..."
5,0.993970,0.994081,0.993997,0.999913,"Bootstrap(n_iterations=5, sample_size=0.8)",PCA(n_components=50),"SVC(kernel='poly',C=8,decision_function_shape=..."


## DT

In [69]:
DT_KF_LDA_df = pd.read_csv("finaldatas_LDA/table_res_kf_dt.csv")
DT_BS_LDA_df = pd.read_csv("finaldatas_LDA/table_res_bs_dt.csv")
DT_KF_PCA_df = pd.read_csv("finaldatas_PCA/table_res_kf_dt.csv")
DT_BS_PCA_df = pd.read_csv("finaldatas_PCA/table_res_bs_dt.csv")

### LDA

In [86]:
DT_KF_LDA_df.drop('Reductor(...)',axis=1) # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Classifier(...)
0,0.309665,0.280576,0.207809,0.826782,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
1,0.289588,0.280433,0.207614,0.826860,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
2,0.257282,0.280139,0.207083,0.826773,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
3,0.961401,0.959732,0.960220,0.987463,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
4,0.957885,0.955928,0.956514,0.988989,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
5,0.952725,0.950800,0.951377,0.990076,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
6,0.970246,0.970012,0.970046,0.984341,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
7,0.964014,0.963671,0.963675,0.986619,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
8,0.958395,0.957906,0.957972,0.988334,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."


In [87]:
DT_BS_LDA_df.drop('Reductor(...)',axis=1) #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Classifier(...)
0,0.301810,0.298574,0.231957,0.827668,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
1,0.332514,0.291512,0.232819,0.813956,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
2,0.274950,0.294025,0.220881,0.836534,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
3,0.951306,0.947938,0.949021,0.981597,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
4,0.945120,0.943116,0.943604,0.983712,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
5,0.944620,0.941453,0.942412,0.983709,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
6,0.957688,0.957206,0.957367,0.977658,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
7,0.950787,0.950598,0.950555,0.980613,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
8,0.945113,0.944099,0.944394,0.982998,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."


In [89]:
DT_KF_LDA_df['Classifier(...)'].values

array(["DecisionTreeClassifier(criterion='gini',min_samples_split=2,max_depth=5)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=10,max_depth=5)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=20,max_depth=5)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=2,max_depth=20)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=10,max_depth=20)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=20,max_depth=20)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=2,max_depth=None)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=10,max_depth=None)",
       "DecisionTreeClassifier(criterion='gini',min_samples_split=20,max_depth=None)"],
      dtype=object)

### PCA

In [90]:
DT_KF_PCA_df.drop('Reductor(...)',axis=1) # KFOLDS

,Precision,Recall,F1,AUC,Validator(...),Classifier(...)
0,0.347006,0.275929,0.244205,0.822319,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
1,0.348222,0.276203,0.244361,0.822053,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
2,0.348761,0.277110,0.245639,0.822262,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
3,0.951992,0.951472,0.951378,0.980211,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
4,0.932661,0.932195,0.931912,0.982483,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
5,0.912605,0.911855,0.911634,0.983150,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
6,0.962365,0.962576,0.962357,0.980468,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
7,0.937223,0.936886,0.936824,0.981845,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."
8,0.917711,0.917001,0.916964,0.982896,"KFold(n_iterations=5, shuffle=True)","DecisionTreeClassifier(criterion='gini',min_sa..."


In [91]:
DT_BS_PCA_df.drop('Reductor(...)',axis=1) #BOOTSTRAP

,Precision,Recall,F1,AUC,Validator(...),Classifier(...)
0,0.356103,0.273522,0.248039,0.822501,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
1,0.342422,0.270106,0.242986,0.814624,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
2,0.351278,0.260135,0.220607,0.815363,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
3,0.926637,0.926268,0.926305,0.963634,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
4,0.908146,0.907971,0.907840,0.967487,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
5,0.887926,0.887666,0.887291,0.970042,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
6,0.930784,0.930807,0.930661,0.963889,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
7,0.912895,0.912842,0.912669,0.968507,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."
8,0.888133,0.887756,0.887541,0.970480,"Bootstrap(n_iterations=5, sample_size=0.8)","DecisionTreeClassifier(criterion='gini',min_sa..."


# seccion de pruebas

# KNN con train test split

In [246]:
# from distutils.command.bdist import show_formats
# from xml.etree.ElementInclude import include
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold
# from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score
# from sklearn.metrics import classification_report
# import numpy as np
# import itertools

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

# def split_KNN(X_, Y_, k_splits_ = 10, n_comps_ = 20, k_ = 3):
#   # kf = KFold(n_splits=k_splits_, random_state=42, shuffle=True)
#   # kf = KFold(n_splits=k_splits_)

#   # split = 0
#   precision = recall = f1 = AUC = 0
#   X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.25, 
#                                                       shuffle=True)
#                                                       # random_state=42)
#   # X_train, X_test, Y_train, Y_test = sign_X_train, sign_X_test,sign_Y_train,sign_Y_test


#     # X_train, X_test = X_[train_index, :], X_[test_index, :] 
#     # Y_train, Y_test = Y_[train_index], Y_[test_index]

#   Scaler = StandardScaler()
#   X_train = Scaler.fit_transform(X_train)
#   X_train = pd.DataFrame(X_train)

#   X_test = Scaler.transform(X_test)
#   X_test = pd.DataFrame(X_test)
#   # print(X_test.head())
#   # return 
#   model = KNN_LDA(n_comps_,k_)
#   model.fit(X=X_train, y=Y_train.ravel())
#   Y_predicted = model.predict(X_test)

#   np.set_printoptions(threshold=300)
#   print(Y_test.ravel())
#   print(Y_predicted)      
#   # metrics

#   precision += precision_score(Y_test, Y_predicted, average="weighted")
#   recall += recall_score(Y_test, Y_predicted, average="weighted")
#   f1 += f1_score(Y_test, Y_predicted,  average="weighted")
#   AUC += roc_auc_score(Y_test, model.predict_proba(X_test),multi_class='ovr')

#   # precision /= k_splits_
#   # recall /= k_splits_
#   # f1 /= k_splits_
#   # AUC /= k_splits_

#   print(f"Final metrics ({0}-Fold, {n_comps_}-LDA vectors, {k_}-NN search)\n", 
#   "Precision:", "{:.2f}".format(precision),
#   " | Recall:", "{:.2f}".format(recall), 
#   " | F1:", "{:.2f}".format(f1),
#   " | AUC:","{:.2f}".format(AUC))

#   return(pd.DataFrame(np.array([[precision, recall, f1, AUC, k_splits_, n_comps_, k_]]),
#                    columns=['Precision', 'Recall', 'F1', 'AUC', 'Folds', 'LDA Vectors', 'NN Search']))

#   # plot_cnf_matrix(Y_test, Y_predicted)


# split_KNN(sign_X, sign_Y, 20, 1, 1)

In [247]:
# from xml.etree.ElementInclude import include
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold
# from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score
# from sklearn.metrics import classification_report
# import numpy as np
# import itertools
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


# from sklearn.model_selection import train_test_split

# def query_get_expressions(kd_tree, x_list_validation, y_list, k=3):
#   d,i = kd_tree.query(x_list_validation,k)
#   expression_list = y_list[i]

#   return d, expression_list.flatten()

# def startingKNN(X_, Y_, k_splits_ = 10, n_comps_ = 20, k_ = 3):
#   # kf = KFold(n_splits=k_splits_, random_state=False, shuffle=False)
  
#   df = pd.read_csv("dataset/sign_mnist_train.csv")
#   df
#   from sklearn.preprocessing import StandardScaler
#   x = df.loc[:,  df.columns != "label"].values
#   y = df.loc[:,["label"]].values

#   SS = StandardScaler()
#   x = SS.fit_transform(x)
#   x = pd.DataFrame(x)

#   lda_model = LDA(n_components=21)
#   lda = lda_model.fit_transform(x,y.ravel())
  
#   kd_tree = spatial.KDTree(lda)

#   df_test = pd.read_csv("dataset/sign_mnist_test.csv")
#   df_test

#   x_test = df_test.loc[:,  df_test.columns != "label"].values
#   y_test = df_test.loc[:,["label"]].values

#   x_test = SS.transform(x_test)
#   x_test = pd.DataFrame(x_test)
#   # print(x_test.head())
#   # return

#   tf_test = lda_model.transform(x_test)

#   y_pred_val = []
#   for x_row in tf_test:
#     # print(x_row)
#     # tf = lda_model.transform([x_row])
#     distances, expressions = query_get_expressions(kd_tree, x_row, y, 6)
#     voted_expression = max(set(expressions), key=lambda x: list(expressions).count(x))
#     y_pred_val.append(voted_expression)

#   print(len(y_test))
#   print(len(y_pred_val))

#   precision = precision_score(y_test, y_pred_val, average="weighted")
#   recall = recall_score(y_test, y_pred_val, average="weighted")
#   f1 = f1_score(y_test, y_pred_val,  average="weighted")
# # tf_test.shape

#   # precision /= k_splits_
#   # recall /= k_splits_
#   # f1 /= k_splits_
#   # AUC /= k_splits_

#   print(f"Final metrics ({0}-Fold, {21}-LDA vectors, {6}-NN search)\n", 
#   "Precision:", "{:.2f}".format(precision),
#   " | Recall:", "{:.2f}".format(recall), 
#   " | F1:", "{:.2f}".format(f1))
  
#   plot_cnf_matrix(y_test, y_pred_val)


# startingKNN(sign_X, sign_Y, 20, 5, 1)

# SVM (Support Vector Machines)


In [22]:
# from sklearn.svm import SVC
# from sklearn.svm import LinearSVC
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.model_selection import train_test_split


# def SVM_function(X_, Y_):
#   X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.25, 
#                                                       shuffle=True)
#   #ovo (ne-versus-one)

#   #reducir dimensionalidad, mapea a 8 vectores
#   lda = LDA(n_components=8)
#   X_train_lda = lda.fit_transform(X_train,Y_train.ravel())
#   X_test_lda = lda.transform(X_test)

#   print(X_train_lda.shape)
  
#   # #Entrenamiento del 
#   # #ovo (one-versus-one)
#   # print("OneVsOne: ")
#   # clf = SVC(decision_function_shape='ovo')
#   # clf.fit(X_train_lda, Y_train.ravel())
#   # Y_predicted = clf.predict(X_test_lda)
#   # print(precision_score(Y_test, Y_predicted, average="macro"))
  
#   # #ovr (one-versus-rest)
#   # print("OneVsRest: ")
#   # clf2 = SVC(decision_function_shape='ovr')
#   # clf2.fit(X_train_lda, Y_train.ravel())
#   # Y_predicted2 = clf2.predict(X_test_lda)
#   # print(precision_score(Y_test, Y_predicted2, average="macro"))

#   #LinearSVC
#   print("LinearSVC: ")
#   lin_clf = svm.LinearSVC()
#   lin_clf.fit(X_train_lda, Y_train.ravel())
#   Y_predicted_linear = lin_clf.predict(X_test_lda)
#   print(precision_score(Y_test, Y_predicted_linear, average="macro"))

# SVM_function(sign_X, sign_Y)



(25970, 8)
LinearSVC: 
0.7391057485130804


C:\Users\SSD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# ARBOLES DE DECISION

In [295]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, recall_score, roc_auc_score, precision_score

# def decisionTree(X,y):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
#     clf = DecisionTreeClassifier()
#     clf = clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)

#     print("precision_score", precision_score(y_test, y_pred, average = 'macro'))
#     print("f1_score", f1_score(y_test, y_pred, average = 'macro'))
#     print("recall_score", recall_score(y_test, y_pred, average = 'macro'))
#     print("roc_auc_score", roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr', average = 'macro'))
    
# decisionTree(sign_X,sign_Y)

precision_score 0.8612534986935819
f1_score 0.8609226131549891
recall_score 0.8610190826864242
roc_auc_score 0.9274861023117129


In [325]:
#Bootstrap
"""import operator
def random_forest(dataset, n):
    forest = []
    test_data = pd.DataFrame(columns=dataset.columns)
    size = len(dataset[PREDICT])
    for _ in range(n):
        idx = np.sort(np.random.choice(size, int(0.7*size), replace=True))
        n_idx = np.array([_ for _ in range(size) if _ not in idx])
        #display(data.iloc[idx, :])
        #display(data.iloc[n_idx, :])
        tr = Tree(get_info_gain)
        tr.train(dataset.iloc[idx, :]) 
        forest.append(tr)
        test_data = pd.concat([test_data, dataset.iloc[n_idx, :]])
    return forest, test_data.drop_duplicates()


def predict(trees, row):
    pred = [t.traversal(t.root,row) for t in trees]
    return max(set(pred), key=pred.count)

def bootstrap(trees, data):
    bool_list = [predict(trees,d) == d[PREDICT] for d in data.to_dict(orient='records')]
    print(bool_list)

f,d = random_forest(df,5)
bootstrap(f,d)"""

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
      
# table_res_bs
# table_res_bs.to_csv("table_res_bs_output.csv",index=False)

# cols = ['Precision', 'Recall', 'F1', 'AUC', 'Validator(...)','Reductor(...)', 'Classifier(...)']
# table_res = pd.DataFrame(columns=cols)

# # BOOTSTRAP

# # K neighbors
# print("KNN Classification")
# for it in [5]:
#   for sample in [0.50]:
#     for vector in [2,8]:
#       for K_ in [1,3,6]:
#         res = Bootstrap_CLF(sign_X, sign_Y, 
#                             n_iterations=it, 
#                             sample_size=sample, 
#                             model=KNeighborsClassifier(n_neighbors=K_),
#                             reducer=LDA(n_components=vector))
#         table_res = pd.concat([table_res, pd.DataFrame([np.append(res, 
#                             [f"Bootstrap(n_iterations={it}, sample_size={sample})",
#                               f"LDA(n_components={vector})",
#                               f"KNeighborsClassifier(n_neighbors={K_})"])],
#                             columns=cols)],axis=0, ignore_index = True)
#         table_res.to_csv("table_res_bootstrap_output.csv",index=False)


# # SVM
# print("SVM Classification")
# for it in [5]:
#   for sample in [0.50]:
#     for vector in [2,8]:
#       for C_ in [0.01, 0.1]: # valores peque;os para dataset grande
#         res = Bootstrap_CLF(sign_X, sign_Y, 
#                             n_iterations=it, 
#                             sample_size=sample, 
#                             model=SVC(kernel='poly', decision_function_shape='ovo', C=C_, probability=True),
#                             reducer=LDA(n_components=vector))
#         table_res = pd.concat([table_res, pd.DataFrame([np.append(res, 
#                             [f"Bootstrap(n_iterations={it}, sample_size={sample})",
#                               f"LDA(n_components={vector})",
#                               f"SVC(kernel='poly',C=C_,decision_function_shape=ovo)"])],
#                             columns=cols)],axis=0, ignore_index = True)
#         table_res.to_csv("table_res_bootstrap_output.csv",index=False)

#         res = Bootstrap_CLF(sign_X, sign_Y, 
#                             n_iterations=it, 
#                             sample_size=sample, 
#                             model=CalibratedClassifierCV(LinearSVC(C=C_,multi_class='ovr',max_iter=10000)),
#                             reducer=LDA(n_components=vector))
#         table_res = pd.concat([table_res, pd.DataFrame([np.append(res, 
#                             [f"Bootstrap(n_iterations={it}, sample_size={sample})",
#                               f"LDA(n_components={vector})",
#                               f"LinearSVC(C={C_}))"])],
#                             columns=cols)],axis=0, ignore_index = True)
#         table_res.to_csv("table_res_bootstrap_output.csv",index=False)


# # Decision trees
# print("Decision Tree Classification")
# for it in [5]:
#   for sample in [0.50]:
#     for vector in [2,8]:
#       for depth in [5,10,20,None]:
#         res = Bootstrap_CLF(sign_X, sign_Y, 
#                             n_iterations=it, 
#                             sample_size=sample, 
#                             model = DecisionTreeClassifier(criterion="gini",max_depth=depth),
#                             reducer=LDA(n_components=vector))
#         table_res = pd.concat([table_res, pd.DataFrame([np.append(res, 
#                             [f"Bootstrap(n_iterations={it}, sample_size={sample})",
#                               f"LDA(n_components={vector})",
#                               f"DecisionTreeClassifier(criterion='gini',max_depth={depth})"])],
#                             columns=cols)],axis=0, ignore_index = True)
#         table_res.to_csv("table_res_bootstrap_output.csv",index=False)
                            
# table_res
# table_res.to_csv("table_res_bootstrap_output.csv",index=False)